# Cart Algorithm From Scratch

In [60]:
import pandas as pd
import numpy as np

### Creating the DF

In [2]:
df = pd.DataFrame()
outlook = ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain']
temp = ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild']
humidity = ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High']
wind = ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong']
decision = [0, 0, 1, 1, 1, 0, 1,0, 1, 1, 1, 1, 1, 0]

In [120]:
df['outlook'] = outlook
df['temp'] = temp
df['humidity'] = humidity
df['wind'] = wind
df['decision'] = decision


In [121]:
df

,outlook,temp,humidity,wind,decision
0,Sunny,Hot,High,Weak,0
1,Sunny,Hot,High,Strong,0
2,Overcast,Hot,High,Weak,1
3,Rain,Mild,High,Weak,1
4,Rain,Cool,Normal,Weak,1
5,Rain,Cool,Normal,Strong,0
6,Overcast,Cool,Normal,Strong,1
7,Sunny,Mild,High,Weak,0
8,Sunny,Cool,Normal,Weak,1
9,Rain,Mild,Normal,Weak,1


### Calaculating the Gini for an Attribute

In [5]:
def calc_gini_for_attribute(class_name, col,df, target_col='decision'):
    total_count = len(df[df[col].isin([class_name])])
    count_of_1 = len(df[(df[col].isin([class_name])) & (df[target_col] == 1)])
    count_of_0 = len(df[(df[col].isin([class_name])) & (df[target_col] == 0)])
    prob_of_1 = count_of_1 / total_count
    prob_of_0 = count_of_0 / total_count
    gini = 1 - (prob_of_1 **2) - (prob_of_0**2)
    return gini, total_count

In [6]:
calc_gini_for_attribute('Sunny', 'outlook',df, target_col='decision')

(0.48, 5)

In [7]:
col = 'outlook'
list(df[col].unique())

['Sunny', 'Overcast', 'Rain']

In [8]:
cols = ['outlook', 'temp', 'humidity', 'wind']
gini_dict = {}
for col in cols:
    print(col)
    gini_for_attr = 0
    for value in list(df[col].unique()):
        gini_val, var_count = calc_gini_for_attribute(value, col, df)
        print(f"For atr: {value}, Value = {gini_val}")
        gini_for_attr += var_count/len(df) * gini_val

    print(round(gini_for_attr, 3))
    print('\n')
    gini_dict[col] = round(gini_for_attr, 3)

outlook
For atr: Sunny, Value = 0.48
For atr: Overcast, Value = 0.0
For atr: Rain, Value = 0.48
0.343


temp
For atr: Hot, Value = 0.5
For atr: Mild, Value = 0.4444444444444445
For atr: Cool, Value = 0.375
0.44


humidity
For atr: High, Value = 0.489795918367347
For atr: Normal, Value = 0.24489795918367355
0.367


wind
For atr: Weak, Value = 0.375
For atr: Strong, Value = 0.5
0.429




In [9]:
gini_dict

{'outlook': 0.343, 'temp': 0.44, 'humidity': 0.367, 'wind': 0.429}

In [10]:
def calc_gini(cols: list, data):
    gini_dict = {}
    for col in cols:
        gini_for_attr = 0
        for value in list(data[col].unique()):
            gini_val, var_count = calc_gini_for_attribute(value, col, data)
            gini_for_attr += var_count/len(data) * gini_val
        gini_dict[col] = round(gini_for_attr, 3)
    return gini_dict

In [11]:
calc_gini(cols, df)

{'outlook': 0.343, 'temp': 0.44, 'humidity': 0.367, 'wind': 0.429}

In [12]:
df['outlook'].values[0]

'Sunny'

In [59]:
list(df.columns)

['outlook', 'day', 'temp', 'humidity', 'wind', 'decision']

### Function to get the best attribute

In [72]:
def get_sel_attr(df):
    cols = list(df.columns)
    attr_gini = calc_gini(cols, df)
    min = 10
    for col in cols:
        if attr_gini[col] < min:
            min = attr_gini[col]
            sel_attr = col
    return sel_attr

### Function to Construct the Tree

In [108]:
# Here we can split the df -> We need to send the selected attribute
def split_df(sel_attr, df, father):
    global id
    print(sel_attr)
    list_of_unique_values = list(df[sel_attr].unique())
    for value in list_of_unique_values:
        if check_termination(df[df[sel_attr] == value]):
            print(f"terminating when {sel_attr} is {value}")        
            id += 1
            final_tree.append({'id': id, 'data': df[df[sel_attr] == value], 'cond': sel_attr + " is " +  value, 'children': [0, 0], 'isRoot': False, 'isLeaf': True, 'father': father})
        else:
            print(f"Cannot terminate when {sel_attr} is {value}")
            id +=1
            new_df = df[df[sel_attr] == value].drop([sel_attr], axis = 1)  
            # print(new_df.head())
            final_tree.append({'id': id, 'data': df[df[sel_attr] == value], 'cond': sel_attr + " is " +  value, 'children': [], 'isRoot': False, 'isLeaf': False, 'father': father})
            new_best_attr = get_sel_attr(new_df)
            print(f"New Attr: {new_best_attr}")
            split_df(new_best_attr, new_df, id)
            
    return {'success': False}
    

In [111]:
# We can terminate if the probability of one class exceeds 75%
def check_termination(df):
    df_length = len(df)
    zero_count = len(df[df['decision'] == 0])
    one_count = len(df[df['decision'] == 1])
    higher = zero_count/df_length if zero_count/df_length > one_count/df_length else one_count/df_length
    print(higher)
    if (higher > 0.9):
        return True
    return False
    

In [118]:
# We define the final tree variable which will store the decision tree
id = 1
final_tree = [{'id': 1, 'data': df, 'cond': 'None', 'children': [], 'isRoot': True, 'isLeaf': False, 'father': 0}]
split_df('outlook', df, id)


outlook
0.6
Cannot terminate when outlook is Sunny
New Attr: humidity
humidity
1.0
terminating when humidity is High
1.0
terminating when humidity is Normal
1.0
terminating when outlook is Overcast
0.6
Cannot terminate when outlook is Rain
New Attr: wind
wind
1.0
terminating when wind is Weak
1.0
terminating when wind is Strong


{'success': False}

## Final Decision Tree

In [119]:
for obj in final_tree:
    print(f"id: {obj['id']} --- cond: {obj['cond']} --- father: {obj['father']}")

id: 1 --- cond: None --- father: 0
id: 2 --- cond: outlook is Sunny --- father: 1
id: 3 --- cond: humidity is High --- father: 2
id: 4 --- cond: humidity is Normal --- father: 2
id: 5 --- cond: outlook is Overcast --- father: 1
id: 6 --- cond: outlook is Rain --- father: 1
id: 7 --- cond: wind is Weak --- father: 6
id: 8 --- cond: wind is Strong --- father: 6
